# Extração de contas dos programas do FNDE

Importando os pacotes necessários

In [1]:
import urllib.request

In [2]:
import re

In [3]:
import pandas as pd

In [4]:
import ssl

Desabilitando a verificação do contexto

In [5]:
context = ssl._create_unverified_context()

Criando a lista para armazenar os anos da pesquisa

In [6]:
anos = []

Definindo o ano inicial da pesquisa

In [7]:
primeiro_ano = 2018

Definindo o ano final da pesquisa

In [8]:
ultimo_ano = 2018

Carregando a lista com os anos a serem pesquisados

In [9]:
for ano in range(primeiro_ano, ultimo_ano + 1):
    anos.append(ano)

Importando o banco de dados com os códigos e os nomes dos programas a serem pesquisados

In [10]:
programas = pd.read_excel('programas.xlsx')

Separando os códigos do banco de dados

In [11]:
codigos = programas['Código']

Definindo o formato da string da url da pesquisa

In [12]:
url = 'https://www.fnde.gov.br/sigefweb/index.php/extratos?ano={}&programa={}&confirmar=Confirmar'

Criando uma lista para armazenar as urls da pesquisa

In [13]:
urls = []

Carregando a lista com as urls a serem pesquisadas

In [14]:
for ano in reversed(anos):
    for codigo in codigos.iteritems():
        urls.append(url.format(ano, codigo[1]))

Conferindo as urls carregadas na lista

In [15]:
urls

['https://www.fnde.gov.br/sigefweb/index.php/extratos?ano=2018&programa=23&confirmar=Confirmar',
 'https://www.fnde.gov.br/sigefweb/index.php/extratos?ano=2018&programa=C7&confirmar=Confirmar',
 'https://www.fnde.gov.br/sigefweb/index.php/extratos?ano=2018&programa=52&confirmar=Confirmar',
 'https://www.fnde.gov.br/sigefweb/index.php/extratos?ano=2018&programa=0C&confirmar=Confirmar',
 'https://www.fnde.gov.br/sigefweb/index.php/extratos?ano=2018&programa=48&confirmar=Confirmar',
 'https://www.fnde.gov.br/sigefweb/index.php/extratos?ano=2018&programa=50&confirmar=Confirmar',
 'https://www.fnde.gov.br/sigefweb/index.php/extratos?ano=2018&programa=TA&confirmar=Confirmar',
 'https://www.fnde.gov.br/sigefweb/index.php/extratos?ano=2018&programa=CV&confirmar=Confirmar',
 'https://www.fnde.gov.br/sigefweb/index.php/extratos?ano=2018&programa=CT&confirmar=Confirmar',
 'https://www.fnde.gov.br/sigefweb/index.php/extratos?ano=2018&programa=40&confirmar=Confirmar',
 'https://www.fnde.gov.br/sige

Criando uma lista para carregar as esferas pesquisadas em cada url

In [16]:
lista_esferas = []

Carregando a lista de esferas a partir das urls

In [17]:
for site in urls:
    with urllib.request.urlopen(site, context=context) as response:
       html = response.read().decode('ansi')
    esferas = re.findall(r'</a></td><td><a href=\"(.*?)\">', html)
    lista_esferas = lista_esferas + esferas

Conferindo a lista de esferas carregada

In [18]:
lista_esferas

['/sigefweb/index.php/extratos/resultado-parcial/ano/2018/programa/23/esfera/municipal',
 '/sigefweb/index.php/extratos/resultado-parcial/ano/2018/programa/C7/esfera/municipal',
 '/sigefweb/index.php/extratos/resultado-parcial/ano/2018/programa/C7/esfera/estadual',
 '/sigefweb/index.php/extratos/resultado-parcial/ano/2018/programa/52/esfera/municipal',
 '/sigefweb/index.php/extratos/resultado-parcial/ano/2018/programa/0C/esfera/municipal',
 '/sigefweb/index.php/extratos/resultado-parcial/ano/2018/programa/48/esfera/municipal',
 '/sigefweb/index.php/extratos/resultado-parcial/ano/2018/programa/48/esfera/estadual',
 '/sigefweb/index.php/extratos/resultado-parcial/ano/2018/programa/50/esfera/particular',
 '/sigefweb/index.php/extratos/resultado-parcial/ano/2018/programa/TA/esfera/municipal',
 '/sigefweb/index.php/extratos/resultado-parcial/ano/2018/programa/CV/esfera/municipal',
 '/sigefweb/index.php/extratos/resultado-parcial/ano/2018/programa/CT/esfera/municipal',
 '/sigefweb/index.php/

Definindo a url do site do FNDE

In [19]:
site_fnde = 'https://www.fnde.gov.br'

Criando a lista das urls das entidades de cada programa

In [20]:
lista_entidades = []

Carregando a lista de entidades de cada programa

In [21]:
for esfera in lista_esferas:
    with urllib.request.urlopen(site_fnde + esfera, context=context) as response:
       html = response.read().decode('ansi')
    entidades = re.findall(r'<tr><td><a href=\"(.*?)\">', html)
    lista_entidades = lista_entidades + entidades

Criando as listas onde serão armazenadas as informações das entidades pesquisadas

In [22]:
lista_cnpjs = []

In [23]:
lista_razoes = []

In [24]:
lista_bancos = []

In [25]:
lista_contas = []

In [26]:
lista_programas = []

Carregando as listas com as informações de cada entidade. As expressões regulares foram construídas com a ferramenta disponível no site: http://www.regexlab.com/wild2regex

In [27]:
for entidade in lista_entidades:    
    with urllib.request.urlopen(site_fnde + entidade, context=context) as response:
        html = response.read().decode('ansi')
    cnpj = re.findall(r'<th>CNPJ\:</th>\s+<td>((?:(?!</td>)(?:.|\n))*)</td>', html, re.MULTILINE)
    lista_cnpjs.append(cnpj)
    razao = re.findall(r'<th>Razão\s+Social\:</th>\s+<td>((?:(?!</td>)(?:.|\n))*)</td>', html, re.MULTILINE)
    lista_razoes.append(razao)
    banco = re.findall(r'<th>Banco\:</th>\s+<td>((?:(?!</td>)(?:.|\n))*)</td>', html, re.MULTILINE)
    lista_bancos.append(banco)
    conta = re.findall(r'<th>Conta\s+Corrente\:</th>\s+<td>((?:(?!</td>)(?:.|\n))*)</td>', html, re.MULTILINE)
    lista_contas.append(conta)
    programa = re.findall(r'<th>Programa\:</th>\s+<td>((?:(?!</td>)(?:.|\n))*)</td>', html, re.MULTILINE)
    lista_programas.append(programa)

RemoteDisconnected: Remote end closed connection without response

Conferindo o conteúdo das listas das informações

In [29]:
len(lista_cnpjs)

4668

In [30]:
len(lista_razoes)

4668

In [31]:
len(lista_bancos)

4619

In [32]:
len(lista_contas)

4619

In [33]:
len(lista_programas)

4668

Criando um dicionário com as informações pesquisadas

In [ ]:
entidades = ['CNPJ': lista_cnpjs, 'RAZÃO SOCIAL': lista_razoes, 'BANCO': lista_bancos, 'CONTA': lista_contas, 'PROGRAMA': lista_programas]

Criando um novo banco de dados a partir do dicionário entidades

In [ ]:
df = pd.DataFrame.from_dict(entidades)